In [1]:
import pandas as pd
import os
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.patches as patches
from sklearn.metrics.pairwise import manhattan_distances
import numpy as np
import scipy.stats as stats

In [2]:
path = '~/Documents/GitHub/S2-classification/Data'

In [3]:
colors = ["#F7F467",
          "#5BF78A",
          "#E1AF7D",
          "#C59360",
          "#FE3DFC",
          "#F77C7F",
          "#59C23A",
          "#00F72C",
          "#466D40",
          "#89E14D",
          "#02424B",
          "#0EFCFE",
          "#F7924D",
          "#BFBFBF",
          #"#F7C557",
          "#FFFD33",
          "#0028FB"]

# C59360 -> barren with shadow

In [4]:
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.figsize'] = 5,5

In [5]:
ndvi = pd.read_csv(os.path.join(path, 'ndvi.csv'), index_col=0)
ndvi['ndvi_pen'] = ndvi['ndvi_pen']/ndvi['n']
ndvi['ndvi_pen'] = (ndvi['ndvi_pen'] - ndvi['ndvi_pen'].min())/(ndvi['ndvi_pen'].max() - ndvi['ndvi_pen'].min())
ndvi.drop(columns=['ID','n'], axis=1, inplace=True)
ndvi = ndvi.sort_values(by=['specie'])

In [6]:
# unique specie
specie = list(set(ndvi['specie']))
print(sorted(specie))

['almond', 'avocado', 'barren', 'barren shadowed', 'forage', 'industrial grape', 'lemon', 'mandarin', 'olive', 'orange', 'riverside vegetation', 'short cycle crop', 'table grape', 'urban', 'walnut', 'water']


In [7]:
ndvi.shape

(2683, 14)

In [8]:
# get the number of  samples by specie
samples = ndvi.groupby('specie').count().iloc[:,0]

In [9]:
# change the name of the first 12 columns following this schema:
# 0 to 8: mayo, 2021 to diciembre, 2021
# 9 to 11: enero, 2022 to abril, 2022

# get the columns names
columns = ndvi.columns.to_list()

# change the name of the columns
columns[0:12] = ['mayo, 2021', 'junio, 2021', 'julio, 2021', 'agosto, 2021', 'septiembre, 2021', 'octubre, 2021', 'noviembre, 2021', 'diciembre, 2021', 'enero, 2022', 'febrero, 2022', 'marzo, 2022', 'abril, 2022']

# set the new columns names
ndvi.columns = columns

In [11]:
#plt.rcParams['figure.figsize'] = 10,3
#for sp in specie:
#    print(sp)
#    # get the samples of the specie
#    specie_samples = ndvi[ndvi['specie'] == sp]
#    # get 10 random samples
#    specie_samples = specie_samples.sample(n=20, random_state=123)
#    # plot the first 12 columns and add random cocolors to each line
#    specie_samples.iloc[:,0:12].T.plot(legend=False)
#    # add horizontal line at y = 0
#    plt.axhline(y=0, color='black', linestyle='--')
#    plt.title(f'Clase: {sp}')
#    plt.ylim(-0.2,1)
#    plt.xlabel('')
#    plt.ylabel('$NDVI$')
#    plt.tight_layout()
#    # rotate the xticks
#    plt.xticks(rotation=0)
#    plt.savefig(f"../document/Thesis/figures/sample_{sp}.pdf", bbox_inches = 'tight')
#    plt.show()


In [ ]:
# create a barplot
plt.rcParams['figure.figsize'] = 10,5
fig, ax = plt.subplots()
ax.bar(range(len(samples)), samples, color=colors)
ax.set_xticks(range(len(samples)))
ax.set_xticklabels(samples.index.to_list(), rotation=90)
ax.set_ylabel('Número de muestras')
ax.set_xlabel('')
ax.set_title('')
plt.tight_layout()
plt.savefig("../document/Thesis/figures/samples.pdf", bbox_inches = 'tight')
plt.show()


In [ ]:
# apply tsne reducing to 3 components
tsne = TSNE(n_components=2, verbose=1, perplexity=50, n_iter=1000, metric='manhattan', random_state=42)
ndvi_tsne = tsne.fit_transform(ndvi.drop(columns=['specie'], axis=1))

ndvi_tsne = pd.DataFrame(ndvi_tsne, columns=['c1','c2'])
ndvi_tsne['specie'] = ndvi.reset_index()['specie']

plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.figsize'] = 10,10
# plot with seaborn
sns.set_context("notebook", font_scale=1.1)
sns.set_style("ticks")
# set size
sns.scatterplot(x='c1', y='c2', hue='specie', style='specie', data=ndvi_tsne, s = 70, alpha = 1, palette=colors, legend="full")
# set legend position
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('t-SNE NDVI')
plt.show()

In [ ]:
pca = PCA(n_components=12)
ndvi_pca = pca.fit_transform(ndvi.drop(columns=['specie'], axis=1))

# convert to dataframe with 12 columns
ndvi_pca = pd.DataFrame(ndvi_pca, columns=[f'c{i+1}' for i in range(12)])
ndvi_pca['specie'] = ndvi.reset_index()['specie']
ndvi_pca['specief'] = pd.factorize(ndvi_pca['specie'])[0]
ndvi_pca['speciec'] = ndvi_pca['specief'].apply(lambda x: colors[x])

# plot the two first components of the PCA with seaborn
sns.scatterplot(x='c1', y='c2', hue='specie', style='specie', data=ndvi_pca, s = 70, alpha = 1, palette=colors, legend="full")
# set legend position
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('PCA NDVI')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = 5,5
concatenated = pd.concat([ndvi_pca.reset_index().assign(dataset='PCA $NDVI$'), ndvi_tsne.reset_index().assign(dataset='t-SNE $NDVI$')])

# set style to specie field
g = sns.FacetGrid(concatenated, hue='specie', col='dataset',height=5, palette=colors, sharex=False, sharey=False, legend_out=True)
g.map(sns.scatterplot,'c1','c2',alpha=1, s=40, linewidth=0.5, edgecolor='white')
g.add_legend()
# change legend main title
new_title = 'Clase'
g._legend.set_title(new_title)
plt.savefig("../document/Thesis/figures/both_ndvi.pdf", bbox_inches = 'tight')
plt.show()

In [ ]:
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 150
plt.rcParams['figure.figsize'] = 5,5

varndvi = np.cumsum(pca.explained_variance_ratio_)

# cum sum of explained variance ratio and add point where it reaches 95%
# plt.plot(np.cumsum(pca.explained_variance_ratio_), marker='o')
# plt.xlabel('Número de componentes')
# plt.ylabel('Varianza explicada acumulada')
# plt.savefig("../document/Thesis/figures/varndvi.pdf", bbox_inches = 'tight')
# plt.show()

In [ ]:
species = ndvi_tsne['specie'].unique()

# create new columns based on specie unique values
for specie in species:
    ndvi_tsne[f'{specie}'] = 0

for row in ndvi_tsne.index:
    temp = ndvi_tsne.loc[row, ['c1','c2']]
    tempspecie = ndvi_tsne.loc[row, 'specie']
    for specie in species:
        temp2 = ndvi_tsne[specie == ndvi_tsne['specie']][['c1','c2']]
        # exclude actual row
        temp2 = temp2[temp2.index != row]
        ndvi_tsne.loc[row, specie] = manhattan_distances(temp.values.reshape(1,-1), temp2.values).min()

# drop c1 and c2 columns, and pivot from almond to water to long format
ndvi_pivoted = ndvi_tsne.drop(columns=['c1','c2'], axis=1).melt(id_vars='specie', var_name='specie2', value_name='distance')

In [ ]:
plt.rcParams['figure.figsize'] = 5,5

# rename specie column to Clase
ndvi_pivoted = ndvi_pivoted.rename(columns={'specie':'Clase', 'distance':'Distancia'})

g = sns.catplot(data=ndvi_pivoted, x='specie2', y='Distancia',
                col='Clase', kind='box', col_wrap=4,
                hue='specie2', palette=colors, height=3.5, aspect=1)

i = 0
for ax in g.axes_dict.items():
    rect = patches.Rectangle((-0.5 + i, 0), 1, 110, linewidth=1, edgecolor='none', facecolor='#F1F1F1')
    ax[1].add_patch(rect)
    if i >= 12:
        ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)
    i = i + 1
g.set_xlabels('')
plt.savefig("../document/Thesis/figures/distndvi.pdf", bbox_inches = 'tight')
plt.show()

In [ ]:
b02 = pd.read_csv(os.path.join(path, 'b02.csv'), index_col=0)
b02['b02_pen'] = b02['b02_pen']/b02['n']
b02['b02_pen'] = (b02['b02_pen'] - b02['b02_pen'].min())/(b02['b02_pen'].max() - b02['b02_pen'].min())
b02.drop(columns=['ID','n','specie'], axis=1, inplace=True)

b03 = pd.read_csv(os.path.join(path, 'b03.csv'), index_col=0)
b03['b03_pen'] = b03['b03_pen']/b03['n']
b03['b03_pen'] = (b03['b03_pen'] - b03['b03_pen'].min())/(b03['b03_pen'].max() - b03['b03_pen'].min())
b03.drop(columns=['ID','n','specie'], axis=1, inplace=True)

b04 = pd.read_csv(os.path.join(path, 'b04.csv'), index_col=0)
b04['b04_pen'] = b04['b04_pen']/b04['n']
b04['b04_pen'] = (b04['b04_pen'] - b04['b04_pen'].min())/(b04['b04_pen'].max() - b04['b04_pen'].min())
b04.drop(columns=['ID','n','specie'], axis=1, inplace=True)

b08 = pd.read_csv(os.path.join(path, 'b08.csv'), index_col=0)
b08['b08_pen'] = b08['b08_pen']/b08['n']
b08['b08_pen'] = (b08['b08_pen'] - b08['b08_pen'].min())/(b08['b08_pen'].max() - b08['b08_pen'].min())
b08.drop(columns=['ID','n'], axis=1, inplace=True)

# Merge all bands
bands = pd.concat([b02,b03,b04,b08], axis=1)
bands = bands.sort_values(by=['specie'])

# apply tsne reducing to 3 components
tsne = TSNE(n_components=2, verbose=1, perplexity=50, n_iter=1000, metric='manhattan', random_state=42)
bands_tsne = tsne.fit_transform(bands.drop(columns=['specie'], axis=1))

bands_tsne = pd.DataFrame(bands_tsne, columns=['c1','c2'])
bands_tsne['specie'] = bands.reset_index()['specie']

plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.figsize'] = 10,10
# plot with seaborn
sns.set_context("notebook", font_scale=1.1)
sns.set_style("ticks")
# set size
sns.scatterplot(x='c1', y='c2', hue='specie', style='specie', data=bands_tsne, s = 70, alpha = 1, palette=colors, legend="full")
# set legend position
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('t-SNE Bands')
plt.show()

In [ ]:
pca = PCA(n_components=51)
bands_pca = pca.fit_transform(bands.drop(columns=['specie'], axis=1))

# convert to dataframe with 12 columns
bands_pca = pd.DataFrame(bands_pca, columns=[f'c{i+1}' for i in range(51)])
bands_pca['specie'] = bands.reset_index()['specie']
bands_pca['specief'] = pd.factorize(bands_pca['specie'])[0]
bands_pca['speciec'] = bands_pca['specief'].apply(lambda x: colors[x])

# plot the two first components of the PCA with seaborn
sns.scatterplot(x='c1', y='c2', hue='specie', style='specie', data=bands_pca, s = 70, alpha = 1, palette=colors, legend="full")
# set legend position
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('PCA Bands')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = 5,5

concatenated = pd.concat([bands_pca.reset_index().assign(dataset='PCA Bandas'), bands_tsne.reset_index().assign(dataset='t-SNE Bandas')])

# set style to specie field
g = sns.FacetGrid(concatenated, hue='specie', col='dataset',height=5, palette=colors, sharex=False, sharey=False, legend_out=True)
g.map(sns.scatterplot,'c1','c2',alpha=1, s=40, linewidth=0.5, edgecolor='white')
g.add_legend()
new_title = 'Clase'
g._legend.set_title(new_title)
plt.savefig("../document/Thesis/figures/both_bands.pdf", bbox_inches = 'tight')
plt.show()

In [ ]:
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 150
plt.rcParams['figure.figsize'] = 5,5

varbands = np.cumsum(pca.explained_variance_ratio_)

# cum sum of explained variance ratio and add point where it reaches 95%
plt.plot(np.cumsum(pca.explained_variance_ratio_), marker='o')
plt.xlabel('Número de componentes')
plt.ylabel('Varianza explicada acumulada')
plt.savefig("../document/Thesis/figures/varbands.pdf", bbox_inches = 'tight')
plt.show()

In [ ]:
species = bands_tsne['specie'].unique()

# create new columns based on specie unique values
for specie in species:
    bands_tsne[f'{specie}'] = 0

for row in bands_tsne.index:
    temp = bands_tsne.loc[row, ['c1','c2']]
    tempspecie = bands_tsne.loc[row, 'specie']
    for specie in species:
        temp2 = bands_tsne[specie == bands_tsne['specie']][['c1','c2']]
        # exclude actual row
        temp2 = temp2[temp2.index != row]
        bands_tsne.loc[row, specie] = manhattan_distances(temp.values.reshape(1,-1), temp2.values).min()

# drop c1 and c2 columns, and pivot from almond to water to long format
bands_pivoted = bands_tsne.drop(columns=['c1','c2'], axis=1).melt(id_vars='specie', var_name='specie2', value_name='distance')

plt.rcParams['figure.figsize'] = 5,5

bands_pivoted = bands_pivoted.rename(columns={'specie':'Clase', 'distance':'Distancia'})

g = sns.catplot(data=bands_pivoted, x='specie2', y='Distancia',
                col='Clase', kind='box', col_wrap=4,
                hue='specie2', palette=colors, height=3.5, aspect=1)

i = 0
for ax in g.axes_dict.items():
    rect = patches.Rectangle((-0.5 + i, 0), 1, 110, linewidth=1, edgecolor='none', facecolor='#F1F1F1')
    ax[1].add_patch(rect)
    if i >= 12:
        ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)
    i = i + 1
g.set_xlabels('')
plt.savefig("../document/Thesis/figures/distbands.pdf", bbox_inches = 'tight')
plt.show()

In [ ]:
# concat bands and ndvi
both = pd.concat([ndvi.drop('specie', axis = 1),bands], axis=1)

# apply tsne reducing to 3 components
tsne = TSNE(n_components=2, verbose=1, perplexity=50, n_iter=1000, metric='manhattan', random_state=42)
both_tsne = tsne.fit_transform(both.drop(columns=['specie'], axis=1))

both_tsne = pd.DataFrame(both_tsne, columns=['c1','c2'])
both_tsne['specie'] = both.reset_index()['specie']

plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.figsize'] = 10,10
# plot with seaborn
sns.set_context("notebook", font_scale=1.1)
sns.set_style("ticks")
# set size
sns.scatterplot(x='c1', y='c2', hue='specie', style='specie', data=both_tsne, s = 70, alpha = 1, palette=colors, legend="full")
# set legend position
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('t-SNE Bands + NDVI')
plt.show()

In [ ]:
pca = PCA(n_components=62)
both_pca = pca.fit_transform(both.drop(columns=['specie'], axis=1))

# convert to dataframe with 12 columns
both_pca = pd.DataFrame(both_pca, columns=[f'c{i+1}' for i in range(62)])
both_pca['specie'] = both.reset_index()['specie']
both_pca['specief'] = pd.factorize(both_pca['specie'])[0]
both_pca['speciec'] = both_pca['specief'].apply(lambda x: colors[x])

# plot the two first components of the PCA with seaborn
sns.scatterplot(x='c1', y='c2', hue='specie', style='specie', data=both_pca, s = 70, alpha = 1, palette=colors, legend="full")
# set legend position
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('PCA Bands + NDVI')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = 5,5

concatenated = pd.concat([both_pca.reset_index().assign(dataset='PCA Bandas + $NDVI$'), both_tsne.reset_index().assign(dataset='t-SNE Bandas + $NDVI$')])

# set style to specie field
g = sns.FacetGrid(concatenated, hue='specie', col='dataset',height=5, palette=colors, sharex=False, sharey=False, legend_out=True)
g.map(sns.scatterplot,'c1','c2',alpha=1, s=40, linewidth=0.5, edgecolor='white')
g.add_legend()
# change legend main title
new_title = 'Clase'
g._legend.set_title(new_title)
plt.savefig("../document/Thesis/figures/both_all.pdf", bbox_inches = 'tight')
plt.show()


In [ ]:
varall = np.cumsum(pca.explained_variance_ratio_)

In [ ]:
species = both_tsne['specie'].unique()

# create new columns based on specie unique values
for specie in species:
    both_tsne[f'{specie}'] = 0

for row in both_tsne.index:
    temp = both_tsne.loc[row, ['c1','c2']]
    tempspecie = both_tsne.loc[row, 'specie']
    for specie in species:
        temp2 = both_tsne[specie == both_tsne['specie']][['c1','c2']]
        # exclude actual row
        temp2 = temp2[temp2.index != row]
        both_tsne.loc[row, specie] = manhattan_distances(temp.values.reshape(1,-1), temp2.values).min()

# drop c1 and c2 columns, and pivot from almond to water to long format
pivoted = both_tsne.drop(columns=['c1','c2'], axis=1).melt(id_vars='specie', var_name='specie2', value_name='distance')

plt.rcParams['figure.figsize'] = 5,5

pivoted = pivoted.rename(columns={'specie':'Clase', 'distance':'Distancia'})

g = sns.catplot(data=pivoted,  x='specie2', y='Distancia',
                col='Clase', kind='box', col_wrap=4,
                hue='specie2', palette=colors, height=3.5, aspect=1)

i = 0
for ax in g.axes_dict.items():
    rect = patches.Rectangle((-0.5 + i, 0), 1, 110, linewidth=1, edgecolor='none', facecolor='#F1F1F1')
    ax[1].add_patch(rect)
    if i >= 12:
        ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)
    i = i + 1
g.set_xlabels('')
plt.savefig("../document/Thesis/figures/distall.pdf", bbox_inches = 'tight')
plt.show()

In [ ]:
# create a list with species as key and color as value
species_color = {}
for i in range(len(species)):
    species_color[species[i]] = colors[i]

In [ ]:
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 150
plt.rcParams['figure.figsize'] = 10,5

plt.plot(varall, marker='.')
plt.plot(varbands, marker='.')
plt.plot(varndvi, marker='.')
# add legend
plt.legend(['PCA Bandas + $NDVI$', 'PCA Bandas', 'PCA $NDVI$'], loc='best')
plt.xlabel('Número de componentes')
plt.ylabel('Varianza explicada acumulada')
plt.savefig("../document/Thesis/figures/variance.pdf", bbox_inches = 'tight')
plt.show()

In [ ]:
# drop c1 and c2 columns, test normality for each species
ndvi_test = ndvi_tsne.drop(columns=['c1','c2'], axis=1)

l = []

for specie in species:
    ndvi_test_temp = ndvi_test[ndvi_test['specie'] == specie]
    for col in ndvi_test_temp.columns[1:]:
        res = stats.normaltest(ndvi_test_temp[col])
        l.append([specie, col, res[0], res[1]])

# convert to dataframe
ndvi_results = pd.DataFrame(l, columns=['specie','band','statistic','pvalue'])

# drop c1 and c2 columns, test normality for each species
ndvi_test = ndvi_tsne.drop(columns=['c1','c2'], axis=1)

ls = []

for specie in species:
    ndvi_test_temp = ndvi_test[ndvi_test['specie'] == specie]
    # compute anova for all groups
    res = stats.tukey_hsd(ndvi_test_temp[species[0]],
                         ndvi_test_temp[species[1]],
                         ndvi_test_temp[species[2]],
                         ndvi_test_temp[species[3]],
                         ndvi_test_temp[species[4]],
                         ndvi_test_temp[species[5]],
                         ndvi_test_temp[species[6]],
                         ndvi_test_temp[species[7]],
                         ndvi_test_temp[species[8]],
                         ndvi_test_temp[species[9]],
                         ndvi_test_temp[species[10]],
                         ndvi_test_temp[species[11]],
                         ndvi_test_temp[species[12]],
                         ndvi_test_temp[species[13]],
                         ndvi_test_temp[species[14]],
                         ndvi_test_temp[species[15]])
    ls.append(res)

ls2 = []

for i in range(len(ls)):
    ls2.append(list(res.pvalue[i,:]))

# convert to dataframe
ndvi_tukey = pd.DataFrame(ls2, columns=species)

# create a heatmap plot from seaborn
sns.heatmap(ndvi_tukey, annot=False, cmap='coolwarm', cbar=False)
# set y ticks values to species
plt.yticks(np.arange(0.5, len(ndvi_tukey.index), 1), species)
# rotate y ticks
plt.yticks(rotation=0)
plt.show()

In [ ]:
bands_test = bands_tsne.drop(columns=['c1','c2'], axis=1)

l = []

for specie in species:
    bands_test_temp = bands_test[bands_test['specie'] == specie]
    for col in bands_test_temp.columns[1:]:
        res = stats.normaltest(bands_test_temp[col])
        l.append([specie, col, res[0], res[1]])

# convert to dataframe
bands_results = pd.DataFrame(l, columns=['specie','band','statistic','pvalue'])

# drop c1 and c2 columns, test normality for each species
bands_test = bands_tsne.drop(columns=['c1','c2'], axis=1)

ls = []

for specie in species:
    bands_test_temp = bands_test[bands_test['specie'] == specie]
    # compute anova for all groups
    res = stats.tukey_hsd(bands_test_temp[species[0]],
                         bands_test_temp[species[1]],
                         bands_test_temp[species[2]],
                         bands_test_temp[species[3]],
                         bands_test_temp[species[4]],
                         bands_test_temp[species[5]],
                         bands_test_temp[species[6]],
                         bands_test_temp[species[7]],
                         bands_test_temp[species[8]],
                         bands_test_temp[species[9]],
                         bands_test_temp[species[10]],
                         bands_test_temp[species[11]],
                         bands_test_temp[species[12]],
                         bands_test_temp[species[13]],
                         bands_test_temp[species[14]],
                         bands_test_temp[species[15]])
    ls.append(res)

ls2 = []

for i in range(len(ls)):
    ls2.append(list(res.pvalue[i,:]))

# convert to dataframe
bands_tukey = pd.DataFrame(ls2, columns=species)

# create a heatmap plot from seaborn
sns.heatmap(bands_tukey, annot=False, cmap='coolwarm', cbar=False)
# set y ticks values to species
plt.yticks(np.arange(0.5, len(bands_tukey.index), 1), species)
# rotate y ticks
plt.yticks(rotation=0)
plt.show()

In [ ]:
both_test = both_tsne.drop(columns=['c1','c2'], axis=1)

l = []

for specie in species:
    both_test_temp = both_test[both_test['specie'] == specie]
    for col in both_test_temp.columns[1:]:
        res = stats.normaltest(both_test_temp[col])
        l.append([specie, col, res[0], res[1]])

# convert to dataframe
both_results = pd.DataFrame(l, columns=['specie','band','statistic','pvalue'])

# drop c1 and c2 columns, test normality for each species
both_test = both_tsne.drop(columns=['c1','c2'], axis=1)

ls = []

for specie in species:
    both_test_temp = both_test[both_test['specie'] == specie]
    # compute anova for all groups
    res = stats.tukey_hsd(both_test_temp[species[0]],
                         both_test_temp[species[1]],
                         both_test_temp[species[2]],
                         both_test_temp[species[3]],
                         both_test_temp[species[4]],
                         both_test_temp[species[5]],
                         both_test_temp[species[6]],
                         both_test_temp[species[7]],
                         both_test_temp[species[8]],
                         both_test_temp[species[9]],
                         both_test_temp[species[10]],
                         both_test_temp[species[11]],
                         both_test_temp[species[12]],
                         both_test_temp[species[13]],
                         both_test_temp[species[14]],
                         both_test_temp[species[15]])
    ls.append(res)

ls2 = []

for i in range(len(ls)):
    ls2.append(list(res.pvalue[i,:]))

# convert to dataframe
both_tukey = pd.DataFrame(ls2, columns=species)

# create a heatmap plot from seaborn
sns.heatmap(both_tukey, annot=False, cmap='coolwarm', cbar=False)
# set y ticks values to species
plt.yticks(np.arange(0.5, len(both_tukey.index), 1), species)
# rotate y ticks
plt.yticks(rotation=0)
plt.show()
